/nfs/homedirs/xzh/equiformer_v2/oc20/trainer/energy_trainer_v2.py  
energy_trainer_v2  

self.normalizer = self.config["dataset"]  

self.use_auxiliary_task = False  

self.name="is2re"  

logger_name = "wandb"


Trainer类中的`self.config`

In [ ]:
self.config = {
    "task": task,  # config["task"]
    "model": model.pop("name"),  #  "equiformer_v2"
    "model_attributes": model,  # the remaining config["model"]
    "optim": optimizer,  # config["optim"]
    "logger": logger,  # "wandb"
    "amp": amp,  # True
    "gpus": distutils.get_world_size() if not self.cpu else 0, # GPU数量，我们计划用1个
    "cmd": {
        "identifier": identifier,  # ""
        "print_every": print_every,  # 10
        "seed": seed,  # 0
        "timestamp_id": self.timestamp_id,  # timestamp, 不再是None
        "commit": commit_hash,  # 不清楚，可能是None
        "checkpoint_dir": os.path.join(
            run_dir, "checkpoints", self.timestamp_id
        ),
        "results_dir": os.path.join(
            run_dir, "results", self.timestamp_id
        ),
        "logs_dir": os.path.join(
            run_dir, "logs", logger_name, self.timestamp_id
        ),
    },
    "slurm": slurm,  # {}
    "noddp": noddp,  # False
}

在 PyTorch 中，`torch.cuda.amp.GradScaler`" 是用于自动混合精度（AMP）训练的一个实用工具。它的作用是帮助在使用浮点16（FP16）精度时维护梯度的大小，以防止在训练过程中出现数值下溢问题。

In [ ]:
self.scaler = torch.cuda.amp.GradScaler()

In [ ]:
self.use_interpolate_init_relaxed_pos = False

In [ ]:
self.grad_accumulation_steps = 1

In [ ]:
@registry.register_logger("wandb")
class WandBLogger(Logger):
    ...

这里`logger_name = 'wandb'`

In [ ]:
self.logger = registry.get_logger_class(logger_name)(self.config)


train method

eval_every: 5000     from config    

primary_metric: self.evaluator.task_primary_metric[self.name]  
这里self.name = 'is2re' -> primary_metric = 'energy_mae'  

self.step: 最初是0  

amp原来是True  

run_dir在sh脚本中自己设定  
在这个命令行脚本中，`--run-dir` 参数指定了一个用于存储运行期间产生的所有输出的目录。这包括训练模型的权重、日志文件、配置文件副本等。具体来说：

- `--run-dir 'models/oc20/s2ef/2M/equiformer_v2/N@12_L@6_M@2/bs@64_lr@2e-4_wd@1e-3_epochs@12_warmup-epochs@0.1_g@8x2'`: 这个参数设置 `run-dir`（运行目录）为一个特定的文件路径。在这种情况下，路径是 `'models/oc20/s2ef/2M/equiformer_v2/N@12_L@6_M@2/bs@64_lr@2e-4_wd@1e-3_epochs@12_warmup-epochs@0.1_g@8x2'`。

这个路径中包含了许多信息，可能反映了用于此次运行的特定配置设置。例如，它可能包含了模型的名称、批大小（bs@64）、学习率（lr@2e-4）、权重衰减（wd@1e-3）、训练周期数（epochs@12）、预热周期（warmup-epochs@0.1）和GPU设置（g@8x2）。这些细节有助于快速识别和引用特定的训练运行和其结果。

总的来说，`run-dir` 在这个脚本中被用作一个组织和存储训练过程输出的地方，使得管理和访问这些输出变得更加方便和直观。   




# 重点！！  

要修改的地方：

* trainer类中，self.dataset的处理

* config中，logger可能需要是一个字典来提供project的信息，  
/nfs/homedirs/xzh/equiformer_v2/ocpmodels/common/logger.py  
@registry.register_logger("wandb")  
class WandBLogger(Logger):  


* 为jarvis数据集注册LmdbDataset类  
/nfs/homedirs/xzh/equiformer_v2/oc20/trainer/lmdb_dataset.py  
相应地，也要修改config.yml下的task.dataset  
**注意还要考虑task的其他键怎么修改**  

* config.yml中的dataset的target_mean, target_std要通过数据集计算出来，  
注意grad_target_mean, grad_target_std有没有被使用到，没有的话就删掉  
The following mean and std are for the dataset in config.yml.
target mean:  tensor(-0.8317)
target std:  tensor(1.0807)

* 检查在计算的过程中，数据集的其他features有没有用到(除了现有的)  
OC20的数据  
Data(pos=[101, 3], cell=[1, 3, 3], atomic_numbers=[101], natoms=101, tags=[101], y=1.9425044000000184, force=[101, 3], fixed=[101], sid=501549, fid=85, id='0_0'), Data(pos=[76, 3], cell=[1, 3, 3], atomic_numbers=[76], natoms=76, tags=[76], y=5.68617614499999, force=[76, 3], fixed=[76], sid=2175174, fid=19, id='0_1'), Data(pos=[32, 3], cell=[1, 3, 3], atomic_numbers=[32], natoms=32, tags=[32], y=1.3902153900000087, force=[32, 3], fixed=[32], sid=1392508, fid=92, id='0_2'), Data(pos=[147, 3], cell=[1, 3, 3], atomic_numbers=[147], natoms=147, tags=[147], y=-1.5692388999999594, force=[147, 3], fixed=[147], sid=2075463, fid=39, id='0_3'), Data(pos=[71, 3], cell=[1, 3, 3], atomic_numbers=[71], natoms=71, tags=[71], y=-0.5425463299999933, force=[71, 3], fixed=[71], sid=1278705, fid=148, id='0_4')